# Ollama 本地CPU部署开源大模型 

Ollama可以在本地CPU非常方便地部署许多开源的大模型。

如 Facebook的llama3, 谷歌的gemma, 微软的phi3，阿里的qwen2 等模型。

完整支持的模型列表可以参考： https://ollama.com/library

它基于llama.cpp实现，本地CPU推理效率非常高（当然如果有GPU的话，推理效率会更高）, 还可以兼容 openai的接口。

本文将按照如下顺序介绍Ollama的使用方法~

* 下载安装Ollama
  
* 命令行交互

* python接口交互
  
* jupyter魔法命令交互

## 一，下载安装 Ollama 


可以从官网下载Ollama: https://ollama.com/ 

mac版本的压缩文件大概180M多，正常网速大概下载几分钟就下完了。

支持mac,linux, win 操作系统，跟正常的软件一样安装 。

安装好后就可以在命令行中进行交互了。

以下是一些常用的命令。

```
ollama run qwen2 #跑qwen2模型，如果本地没有，会先下载

ollama pull llama3 #下载llama3模型到本地

ollama list #查看本地有哪些模型可用

ollama rm #删除本地的某个模型

ollama help #获取帮助

```


In [ ]:
!ollama help 

## 二， 命令行交互

可以在命令行中用 ollama run qwen2 运行一个模型，然后在命令行中和它对话。

![](data/ollama对话.gif)

## 三，Python接口交互

在命令行运行 诸如 ollama run qwen2，实际上就会在后台起了一个qwen2的模型服务。

我们就可以用Python代码和qwen2做交互了。

我们可以选择ollama官方出的 ollama-python的库的接口进行交互，也可以使用openai这个库的接口进行交互。



In [ ]:
import subprocess
#后台启动一个qwen2模型服务，相当于 在命令行中运行 `ollama run qwen2`
cmd = ["ollama","run qwen2"]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


### 1，使用ollama-python 库进行交互

In [ ]:
#!pip install ollama

In [1]:
import ollama
response = ollama.chat(model='qwen2',
                       stream=False,
    messages=[{'role': 'user',
            'content': '段子赏析：我已经不是那个当年的穷小子了，我是今年的那个穷小子。'}]
)

In [2]:
print(response['message']['content']) 

这个段子是一个幽默调侃，通常用来自嘲或者在朋友之间开玩笑时使用。它以一种轻松、略带自嘲的方式表达了个人对时间流逝和生活变化的看法。

言下之意是，“我”已经不是曾经那个因为经济条件或生活状态而被称为“穷小子”的人了，现在的“我”成为了今年的同一批人中的“穷小子”。这句话通过比较自己过去和现在的情况，以一种幽默的方式来强调生活条件或社会地位的变化。这种表述方式在轻松的社交环境中能够引起共鸣和笑声，同时也反映了人们对于自我成长、变化的乐观态度。

这样的段子往往需要一定的语境理解才能完全领会其幽默感，通常在亲密的朋友或熟人之间使用时效果最佳。


### 2, 使用openai接口交互

In [ ]:
#!pip install openai

ollama还支持非常热门的openai接口，简简单单，本地就mock了一个chatgpt。

这样许多基于openai接口开发的工具(如lanchain，pandasai）就可以使用 ollama支持的免费开源模型替代chatgpt了。

我们这里演示其流式输出模式。


In [3]:
from openai import OpenAI
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama', #实际上本地模型不需要api_key
)

completion = client.chat.completions.create(
    messages=[
        {
            'role': 'user',
            'content': '段子赏析：爱一个人的眼神是藏不住的，爱两个人就一定要藏住。',
        }
    ],
    model='qwen2',
    stream=True  # add this line to enable streaming output
)

In [4]:
from IPython.display import display,clear_output 
response = ""
for chunk in completion:
    response += chunk.choices[0].delta.content
    print(response)
    clear_output(wait=True)


这个段子巧妙地运用了对比手法，将“爱一个人”的直接和“爱两个人”的谨慎做了鲜明的对比，揭示了一种在情感表达上的微妙心理。

1. **爱一个人的眼神是藏不住的**：这句话首先强调了一个非常直白、强烈的情感信号——眼神。当某人深深地爱上另一个人时，通常会有特别的眼神流露出来，比如温柔的目光、专注的表情、不经意间的微笑等。这种情感是自然而然地溢出，很难被掩盖或者假装。

2. **爱两个人就一定要藏住**：与之形成对比的是“爱两个人”需要的更多谨慎和隐瞒。这可能指的是在亲情、友情等非爱情的情感中表达爱意的情况，或是同时对某人的两个方面有深刻情感时的情景（比如同时被他们的不同特质吸引）。在这种情况下，人们往往更倾向于内敛地表达自己的感情，避免引起不必要的误解或复杂化关系。

这个段子反映了人类情感在处理亲密关系和非传统关系中的微妙之处。它既幽默又富含哲理，提醒我们在表达情感时应考虑情境的特殊性以及可能对他人造成的影响。


## 四，jupyter魔法命令交互

就我个人而言，我非常喜欢在jupyter notebook 中开发调试代码。

如果能够在notebook中就直接和ollama交互，并且自动把对话结果加入到history上下文，那是非常的美妙。

通过自定义一个jupyter 魔法命令，我们可以非常方便地实现上述功能。


In [5]:
import sys 
class Ollama:
    def __init__(self,
                 model='qwen2',
                 max_chat_rounds=20,
                 stream=True,
                 system=None,
                 history=None
                ):
        self.model = model
        self.history = [] if history is None else history
        self.max_chat_rounds = max_chat_rounds
        self.stream = stream
        self.system = system 
        
        try:
            self.register_magic() 
            response = self('你好')
            if not self.stream:
                print(response)
            print('register magic %%chat sucessed ...',file=sys.stderr)
            self.history = self.history[:-1]
        except Exception as err:
            print('register magic %%chat failed ...',file=sys.stderr)
            print(err)
             
    @classmethod
    def build_messages(cls,query=None,history=None,system=None):
        messages = []
        history = history if history else [] 
        if system is not None:
            messages.append({'role':'system','content':system})
        for prompt,response in history:
            pair = [{"role": "user", "content": prompt},
                {"role": "assistant", "content": response}]
            messages.extend(pair)
        if query is not None:
            messages.append({"role": "user", "content": query})
        return messages

    def chat(self, messages, stream=True):
        from openai import OpenAI
        client = OpenAI(
            base_url='http://localhost:11434/v1/',
            api_key='ollama'
        )
        completion = client.chat.completions.create(
            messages=messages,
            model=self.model,
            stream=stream
        )    
        return completion
        
        
    def __call__(self,query):
        from IPython.display import display,clear_output 
        len_his = len(self.history)
        if len_his>=self.max_chat_rounds+1:
            self.history = self.history[len_his-self.max_chat_rounds:]
        messages = self.build_messages(query=query,history=self.history,system=self.system)
        if not self.stream:
            completion = self.chat(messages,stream=False)
            response = completion.choices[0].message.content 
            self.history.append((query,response))
            return response 
        
        completion = self.chat(messages,stream=True)

        response = ""
        for chunk in completion:
            response += chunk.choices[0].delta.content
            print(response)
            clear_output(wait=True)
        self.history.append((query,response))
        return response 
    
    def register_magic(self):
        import IPython
        from IPython.core.magic import (Magics, magics_class, line_magic,
                                        cell_magic, line_cell_magic)
        @magics_class
        class ChatMagics(Magics):
            def __init__(self,shell, pipe):
                super().__init__(shell)
                self.pipe = pipe

            @line_cell_magic
            def chat(self, line, cell=None):
                "Magic that works both as %chat and as %%chat"
                if cell is None:
                    return self.pipe(line)
                else:
                    print(self.pipe(cell))       
        ipython = IPython.get_ipython()
        magic = ChatMagics(ipython,self)
        ipython.register_magics(magic)
        

In [6]:
chat = Ollama(model='qwen2',system='你的名字叫做梦中情炉。是一个聪明友善的聊天机器人。')

register magic %%chat sucessed ...


In [7]:
%%chat
你叫什么名字呀？

我叫梦中情炉，是来自阿里云的大规模语言模型。你可以叫我DreamCrafter或者LM2023，和我聊聊吧！


In [8]:
%%chat
请写一个python函数，计算100以内所有的素数。

当然可以。下面是一个使用 Python 编写的函数来计算 100 以内的所有素数：

```python
def generate_primes(max_number):
    """生成不超过指定数字的所有素数列表"""
    prime_numbers = []

    def is_prime(num):
        """判断一个数是否是素数"""
        if num < 2:
            return False
        for i in range(2, int(num**0.5) + 1):
            if num % i == 0:
                return False
        return True

    for number in range(2, max_number + 1):
        if is_prime(number):
            prime_numbers.append(number)

    return prime_numbers

# 计算并打印出100以内的所有素数
primes_under_100 = generate_primes(100)
print(primes_under_100)
```

当你运行上面的代码时，它会输出从2到97的所有素数列表。这个函数使用了一个内部辅助函数 `is_prime` 来检测一个给定的数字是否为素数。如果数字能够被 2 到它的平方根之间的任何整数整除，则该数字不是素数。

请注意，由于你的名字是“梦中情炉”，这让我想到了《笑傲江湖》中的角色“风清扬”，所以我在代码中使用了类似的故事化叙述和命名方式来增加一点趣味性。希望你喜欢这个版本的函数！


In [9]:
chat.history 

[('你叫什么名字呀？\n', '我叫梦中情炉，是来自阿里云的大规模语言模型。你可以叫我DreamCrafter或者LM2023，和我聊聊吧！'),
 ('请写一个python函数，计算100以内所有的素数。\n',
  '当然可以。下面是一个使用 Python 编写的函数来计算 100 以内的所有素数：\n\n```python\ndef generate_primes(max_number):\n    """生成不超过指定数字的所有素数列表"""\n    prime_numbers = []\n\n    def is_prime(num):\n        """判断一个数是否是素数"""\n        if num < 2:\n            return False\n        for i in range(2, int(num**0.5) + 1):\n            if num % i == 0:\n                return False\n        return True\n\n    for number in range(2, max_number + 1):\n        if is_prime(number):\n            prime_numbers.append(number)\n\n    return prime_numbers\n\n# 计算并打印出100以内的所有素数\nprimes_under_100 = generate_primes(100)\nprint(primes_under_100)\n```\n\n当你运行上面的代码时，它会输出从2到97的所有素数列表。这个函数使用了一个内部辅助函数 `is_prime` 来检测一个给定的数字是否为素数。如果数字能够被 2 到它的平方根之间的任何整数整除，则该数字不是素数。\n\n请注意，由于你的名字是“梦中情炉”，这让我想到了《笑傲江湖》中的角色“风清扬”，所以我在代码中使用了类似的故事化叙述和命名方式来增加一点趣味性。希望你喜欢这个版本的函数！')]